<a href="https://colab.research.google.com/github/Tabs2206/Meteor-Detection/blob/main/cnnmodelyolov5/YOLOV5_with_CNN_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  # Import PyTorch
import torch.nn as nn  # Import the neural network module
import torchvision.models as models  # Import the models module from torchvision

# Load a pre-trained ResNet model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Remove the final classification layer
feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1]))

# Test with a random image of size 860x538
input_data = torch.randn(1, 3, 538, 860)  # (batch_size, channels, height, width)
features = feature_extractor(input_data)
print(features.shape)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 209MB/s]


torch.Size([1, 512, 1, 1])


In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt  # Install requirements


Cloning into 'yolov5'...
remote: Enumerating objects: 16867, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 16867 (delta 14), reused 28 (delta 4), pack-reused 16825 (from 1)
Receiving objects: 100% (16867/16867), 15.60 MiB | 13.31 MiB/s, done.
Resolving deltas: 100% (11563/11563), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 466.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


## Changes in Architecture
#### In order to add feature extraction we need to perform some changes in YOLOV5 Architecture

We need to change the YAML file contant to the below in order to perform feature extraction.




```
# Ultralytics YOLOv5 🚀, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]],  # Example backbone conv layers
    [-1, 1, Conv, [256, 3, 2]],
    [-1, 3, C3, [256]],
    [-1, 1, Conv, [256, 3, 2]],  # Example P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]],  # Example P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]],  # Example P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]],     # Example SPPF
  ]

head: [
    # x1 and x2 processing and concatenation
    [-1, 1, Conv, [256, 1, 1]],  # x1 Conv: 256 channels
    [-1, 1, Conv, [192, 1, 1]],  # x2 Conv: 192 channels
    
    # Concatenate
    [[-1, 6], 1, Concat, [1]],   # Concatenate x1 and x2 -> 448 channels
    
    # Post-concat Conv to match the next layer's requirements
    [-1, 1, Conv, [512, 1, 1]],  # Adjust to 512 channels
    
    # Continue with upsampling or other operations
    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
    [[-1, 4], 1, Concat, [1]],   # Further concatenation
    [-1, 3, C3, [256, 128, 1, False]],  # Processing continues
]



```
Once the changes is done the the yolov5s.yaml file and run the below code. Make sure the path of the file is correctly set.


In [3]:
import torch
import torch.nn as nn
from torchvision import models
from yolov5.models.yolo import Model

# Custom CNN model definition
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet50(pretrained=True)
        # Use all layers except the last fully connected layer
        self.feature_extractor = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a 1x1 convolution to adjust the number of output channels to 128
        self.channel_adjust = nn.Conv2d(2048, 256, kernel_size=1)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.channel_adjust(x)  # Adjust channels to match YOLOv5 expectations
        return x

# Define a new YOLOv5 model with a custom backbone
class YOLOv5CustomBackbone(Model):
    def __init__(self, cfg='/content/yolov5/models/yolov5s.yaml', ch=256, nc=1):
        super().__init__(cfg, ch, nc)
        # Replace the default backbone with your custom CNN
        self.model[0] = CustomCNN()  # Assuming the first layer is the backbone
        # Initialize 'self.d' for the 'Concat' module in YOLOv5
        self.model[-1].d = -1 # Set the concatenation dimension to the last dimension

    def forward(self, x):
        x1 = self.model[6](x)
        x2 = self.model[-1](x)
        print("Shape of x1 (layer 6 output):", x1.shape)
        print("Shape of x2 (current layer output):", x2.shape)
        x = torch.cat([x1, x2], dim=1)
        return x
# Instantiate the model
model = YOLOv5CustomBackbone()

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1    294976  models.common.Conv                      [256, 32, 6, 2, 2]            
  1                -1  1     37120  models.common.Conv                      [32, 128, 3, 2]               
  2                -1  1     74496  models.common.C3                        [128, 128, 1]                 
  3                -1  1    147712  models.common.Conv                      [128, 128, 3, 2]              
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

# Include a YAML

For training the model in our own classifiation we need to add a new YAML file containing the below details.



```
train: /content/drive/MyDrive/class_meteor/images/train # path to training images
val: /content/drive/MyDrive/class_meteor/images/val  # path to validation images
test: /content/drive/MyDrive/class_meteor/images/test  # path to test images (optional)


nc: 1  # number of classes
names: ['meteor']  # class names
```

This will contain the path of the labelled data set for training, validation and test and the class i.e, meteor in our case.

Drag and drop the file in the file path "/content/yolov5/data"

In the below code we have utilse is under --data to get the training data for training our(train.py)

In [4]:
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 200 --data /content/yolov5/data/coco128_meteor.yaml --weights yolov5s.pt --name meteor

2024-08-18 22:47:31.639647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 22:47:31.659799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 22:47:31.665714: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/coco128_meteor.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scal

### Creating a directory to save the model

In [ ]:
#!mkdir -p /content/drive/MyDrive/cnnyolov5_meteor

In [7]:
save_path = '/content/drive/MyDrive/cnnyolov5_meteor'

In [14]:
!cp /content/yolov5/runs/train/meteor3/weights/best.pt {save_path}/best.pt
!cp /content/yolov5/runs/train/meteor3/weights/last.pt {save_path}/last.pt

In [15]:
!cp -r /content/yolov5/runs/train/meteor3 {save_path}/traindata

## Detecting our model on unseen data.

In [10]:
# Load the model
!python detect.py --weights /content/yolov5/runs/train/meteor/weights/best.pt --img 640 --conf 0.4 --source /content/drive/MyDrive/class_multi/images/val/RAD_BEDOUR_20240619_0035_BELEUV_SYS001.png
# change the --source 'image.png' as per your test image.


detect: weights=['/content/yolov5/runs/train/meteor/weights/best.pt'], source=/content/drive/MyDrive/class_multi/images/val/RAD_BEDOUR_20240619_0035_BELEUV_SYS001.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/class_multi/images/val/RAD_BEDOUR_20240619_0035_BELEUV_SYS001.png: 416x640 10 meteors, 77.2ms
Speed: 0.8ms pre-process, 77.2ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [11]:
# Load the model
!python detect.py --weights /content/yolov5/runs/train/meteor/weights/best.pt --img 640 --conf 0.4 --source /content/drive/MyDrive/RAD_BEDOUR_20240610_0200_BEHUMA_SYS002.png


detect: weights=['/content/yolov5/runs/train/meteor/weights/best.pt'], source=/content/drive/MyDrive/RAD_BEDOUR_20240610_0200_BEHUMA_SYS002.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/RAD_BEDOUR_20240610_0200_BEHUMA_SYS002.png: 416x640 10 meteors, 78.0ms
Speed: 0.8ms pre-process, 78.0ms inference, 1.1ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


In [16]:
!cp /content/yolov5/runs/detect/exp2/RAD_BEDOUR_20240610_0200_BEHUMA_SYS002.png {save_path}/output
!cp /content/yolov5/runs/detect/exp/RAD_BEDOUR_20240619_0035_BELEUV_SYS001.png {save_path}/output1